<a href="https://colab.research.google.com/github/skywalker0803r/mxnet_course/blob/master/mxnet_gluon_linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!pip install mxnet
from mxnet import autograd,nd

num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = nd.random.normal(scale=1, shape=(num_examples, num_inputs))
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += nd.random.normal(scale=0.01, shape=labels.shape)

# batch data

In [0]:
from mxnet.gluon import data as gdata

batch_size = 10
# 将训练数据的特征和标签组合
dataset = gdata.ArrayDataset(features, labels)
# 随机读取小批量
data_iter = gdata.DataLoader(dataset, batch_size, shuffle=True)

In [3]:
for X, y in data_iter:
    print(X, y)
    break


[[-1.0940087   0.9465744 ]
 [-0.9924796   1.1233943 ]
 [-0.61924976 -1.2270768 ]
 [-0.96521896  0.43817237]
 [-0.00357558  0.43679187]
 [-1.0267551   0.82085365]
 [-1.95122    -1.0635319 ]
 [ 0.542846    0.19992658]
 [ 0.82249576  0.68154025]
 [-0.42326602 -0.60970175]]
<NDArray 10x2 @cpu(0)> 
[-1.2060806  -1.6139268   7.137101    0.77646625  2.705378   -0.65667355
  3.9271483   4.6108565   3.5357232   5.427304  ]
<NDArray 10 @cpu(0)>


# define model

In [0]:
from mxnet.gluon import nn
from mxnet import init

net = nn.Sequential()
net.add(nn.Dense(1))
net.initialize(init.Normal(sigma=0.01))

# define loss

In [0]:
from mxnet.gluon import loss as gloss

loss = gloss.L2Loss()  # 平方损失又称L2范数损失

# define trainer

In [0]:
from mxnet import gluon

trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.03})

# trainning

In [7]:
num_epochs = 5
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter: #batch data
        with autograd.record():
            l = loss(net(X), y) #forward and count loss 
        l.backward() # loss backward
        trainer.step(batch_size) # update weight & bias
    l = loss(net(features), labels)
    print('epoch %d, loss: %f' % (epoch, l.mean().asnumpy()))

epoch 1, loss: 0.040746
epoch 2, loss: 0.000154
epoch 3, loss: 0.000050
epoch 4, loss: 0.000050
epoch 5, loss: 0.000050


In [8]:
dense = net[0]
true_w, dense.weight.data()

([2, -3.4], 
 [[ 1.9998696 -3.4003716]]
 <NDArray 1x2 @cpu(0)>)

In [9]:
true_b, dense.bias.data()

(4.2, 
 [4.2005677]
 <NDArray 1 @cpu(0)>)

In [11]:
dense.weight.grad()


[[ 0.009409   -0.00663071]]
<NDArray 1x2 @cpu(0)>